In [1]:
import json

import bw2data

from enbios import Experiment

config = json.load(open("non_linear_config.json", encoding="utf-8"))

In [2]:
exp = Experiment(config)
exp

Excluding 0 filtered results
Excluding 104 filtered results


Experiment: (call info() for details)
Structural nodes: 1
Methods: 1
Hierarchy (depth): 2
Scenarios: 1

In [3]:
bw_adapter_def: dict = config["adapters"][0]
bw_method = bw_adapter_def["methods"]
bw_method

{'GWP1000': ['ReCiPe 2016 v1.03, midpoint (E)',
  'climate change',
  'global warming potential (GWP1000)']}

In [4]:
result = exp.run()
result[Experiment.DEFAULT_SCENARIO_NAME]["results"]

2024-06-05 12:54:07,318 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-05 12:54:15,342 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 1.570765820650225}}

Let's have a look at the characterization factors. They can be loaded from brightway like this.
Whay we are getting back is a list of tuples. Each tuple has two elements, first a list which identifies the biosphere activity by its database and code, and 2nd the characterization factor for that activity.  `([<database>,<code>], <cf>)`.



In [5]:
method_cfs = bw2data.Method(tuple(bw_method["GWP1000"])).load()
# or # bw_adapter.methods["GWP1000"].id
method_cfs

[(['biosphere3', 'e259263c-d1f1-449f-bb9b-73c6d0a32a00'], 1),
 (['biosphere3', '16eeda8a-1ea2-408e-ab37-2648495058dd'], 1),
 (['biosphere3', 'aa7cac3a-3625-41d4-bc54-33e2cf11ec46'], 1),
 (['biosphere3', '349b29d1-3e58-4c66-98b9-9d1a076efd2e'], 1),
 (['biosphere3', 'f9749677-9c9f-4678-ab55-c607dfdc2cb9'], 1),
 (['biosphere3', 'e1c597cc-14cb-4ebb-af07-7a93a5b77d34'], 1),
 (['biosphere3', '6d89125e-e9b7-4d7e-a1fc-ada45dbd8815'], 1),
 (['biosphere3', '78eb1859-abd9-44c6-9ce3-f3b5b33d619c'], 1),
 (['biosphere3', 'e4e9febc-07c1-403d-8d3a-6707bb4d96e6'], 1),
 (['biosphere3', 'e8787b5e-d927-446d-81a9-f56977bbfeb4'], 1),
 (['biosphere3', '28e1e2d6-97ad-4dfd-932a-9edad36dcab9'], 1),
 (['biosphere3', '4e1f0bb0-2703-4303-bf86-972d810612cf'], 1),
 (['biosphere3', 'd6235194-e4e6-4548-bfa3-ac095131aef4'], 1),
 (['biosphere3', 'eba59fd6-f37e-41dc-9ca3-c7ea22d602c7'], 1),
 (['biosphere3', '73ed05cc-9727-4abf-9516-4b5c0fe54a16'], 1),
 (['biosphere3', '259cf8d6-6ea8-4ccf-84b7-23c930a5b2b3'], -1),
 (['bio

Without going into details of what we are changing, we will change one biosphere cf and recalculate 

In [6]:
non_linear_config = bw_adapter_def["config"]["nonlinear_characterization"]
non_linear_config

{'methods': {'GWP1000': {'get_defaults_from_original': True, 'functions': {}}}}

In [7]:
method_cfs[0][0]

['biosphere3', 'e259263c-d1f1-449f-bb9b-73c6d0a32a00']

The assignment of functions to activities, works similar to how brightway does it. `functions` is a dictionary with activity identifiers as keys (tuples!), of database and code (note lists do not work as dictionary keys!). the value can be any type of function, that takes one float variable as input and returns another float.

In [8]:
# we can directly pass functions to 
def cf_double(v: float) -> float:
    return v * 2


non_linear_config["methods"]["GWP1000"]["functions"] = {
    tuple(method_cfs[0][0]): cf_double
}

In [9]:
bw_adapter_def["config"]["nonlinear_characterization"]

{'methods': {'GWP1000': {'get_defaults_from_original': True,
   'functions': {('biosphere3',
     'e259263c-d1f1-449f-bb9b-73c6d0a32a00'): <function __main__.cf_double(v: float) -> float>}}}}

Run this...

In [10]:
Experiment(config).run()[Experiment.DEFAULT_SCENARIO_NAME]["results"]

Excluding 0 filtered results
Excluding 104 filtered results
2024-06-05 12:54:28,592 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-05 12:54:39,906 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 1.5707663153907774}}

In case, that the function can be put into one single expression (generally, one line) one can also use anonymous functions - in python called lambda functions (See https://docs.python.org/3/reference/expressions.html#lambda for details). So the above part could also be expressed with:

In [11]:
non_linear_config["methods"]["GWP1000"]["functions"] = {
    tuple(method_cfs[0][0]): lambda v: v * 2
}

## Using modules (python files)

Since functions can not be included in json files and therefor not be part of an experiment configuration file, it is also possible to refer to functions that defined in other modules, that can sit next to the config file.\nThis is done by including a  `module_path_function_name` for the impact method. The value must be a tuple of 2 strings. The first is the path of the module and the 2nd the name of the function. This function does not take any parameter and must return a dictionary, which must have the same form as the `functions` variable explained before. A dictionary mapping activity ids to functions.  

In [12]:
non_linear_config["methods"]["GWP1000"]["module_path_function_name"] = (
"../data/bw_test_non_linear_methods_module.py", "wpg_1000")

Experiment(config).run()[Experiment.DEFAULT_SCENARIO_NAME]["results"]

Excluding 0 filtered results
Excluding 104 filtered results
2024-06-05 12:54:48,514 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-05 12:54:48,517 - /home/ra/projects/enbios/enbios/bw2/brightway_experiment_adapter.py - WARNING - method functions dict for method: 'GWP1000' is not empty, but function will be overwritten by functions received from 'module_path_function_name'
2024-06-05 12:54:56,150 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 1.5707663153907774}}

This is how the module looks like:

In [13]:
print(open("../data/bw_test_non_linear_methods_module.py").read())

from typing import Callable

def gwp_cf_double(v: float)-> float:
    return v * 2


def wpg_1000() -> dict[tuple[str, str], Callable[[float], float]]:
    return {("biosphere3",'e259263c-d1f1-449f-bb9b-73c6d0a32a00'): gwp_cf_double}



## The complete config:

The configuration for `nonlinear_characterization` in the brightway adapter configuration has only one key: `methods` which includes a non-linear configuration for each respective impact method. Methods which are not included here, will do the normal brightway calculation.

The configuration for each method contains these parameters:  
- functions: dictionary with 2 string tuples as keys (database, code) and functions as values
- module_path_function_name: a tuple of two strings: module_path and function-name. result will be assigned to `functions`
- get_defaults_from_original: Whether all non-defined biosphere activities should use their characterization factor from the original brightway method for a linear function: v * cf. (default: False)
- name: name of the method (will be set by enbios)

## A reduced example
Creating a tiny new system in order to comprehend the calculations. We will create a new database a new activity and connect it to carbon_dioxide (which is the first biosphere activity in the cf array.


In [14]:
from bw2data.backends import Activity

# let's get the actual activity
carbon_dioxide: Activity = bw2data.Database("biosphere3").get(method_cfs[0][0][1])
carbon_dioxide

'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term'))

Let's write this function to clean up afterward

In [15]:
def delete_exchanges_and_db():
    if not "temporary" in bw2data.databases:
        return
    new_db = bw2data.Database("temporary")
    for activity in new_db:
        for bioflow in activity.biosphere():
            bioflow.delete()
    del bw2data.databases["temporary"]

... and create the new system

In [16]:
delete_exchanges_and_db()
new_db = bw2data.Database("temporary")
new_db.register()
code_name = "new_temp_activity"
activity = new_db.new_activity(code_name, name=code_name, unit="unit")
activity.save()
exc = activity.new_exchange(input=carbon_dioxide, amount=1, type="biosphere")
exc.save()

We need to change the hierarchy in order to use this new activity

In [17]:
config["hierarchy"]["children"][0]["config"] = {"code": code_name, "default_output": {"unit": "unit", "magnitude": 1}}

Running this example will result in 2.0 kg CO2-Eq because our functional unit is 1 (default output) the exchange has an amount of one, and as created before, we have a custom characterization function, which doubles the value.

In [18]:
exp = Experiment(config)
exp.run()[Experiment.DEFAULT_SCENARIO_NAME]["results"]

2024-06-05 12:55:04,838 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-05 12:55:04,840 - /home/ra/projects/enbios/enbios/bw2/brightway_experiment_adapter.py - WARNING - method functions dict for method: 'GWP1000' is not empty, but function will be overwritten by functions received from 'module_path_function_name'
2024-06-05 12:55:04,940 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 2.0}}

Doubling the amount of the exchange, will double the result

In [19]:
exc["amount"] = 2
exc.save()
exp.run()[Experiment.DEFAULT_SCENARIO_NAME]["results"]

2024-06-05 12:55:04,958 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-05 12:55:04,961 - /home/ra/projects/enbios/enbios/bw2/brightway_experiment_adapter.py - WARNING - method functions dict for method: 'GWP1000' is not empty, but function will be overwritten by functions received from 'module_path_function_name'
2024-06-05 12:55:05,074 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 4.0}}

And again, we can do any arbitrary calculation... 

In [20]:
from math import pi, log

exc["amount"] = 1
exc.save()


def accum_harmonic_tresh(v: float) -> float:
    if v < 1:
        return 1
    else:
        return log(v * pow(pi, 2))


non_linear_config["methods"]["GWP1000"]["functions"] = {
    tuple(method_cfs[0][0]): accum_harmonic_tresh
}

if "module_path_function_name" in non_linear_config['methods']['GWP1000']:
    del non_linear_config['methods']['GWP1000']['module_path_function_name']
exp = Experiment(config)
exp.run()[Experiment.DEFAULT_SCENARIO_NAME]["results"]

2024-06-05 12:55:06,126 - demos.brightway_adapter_demos.enbios.base - INFO - Running scenario 'default scenario'
2024-06-05 12:55:06,231 - enbios.bw2.MultiLCA_util - DEBUG - Demand 0/1


{'GWP1000': {'unit': 'kg CO2-Eq', 'magnitude': 2.2894597716988003}}

And finally, we clean up

In [21]:
delete_exchanges_and_db()